In [1]:
from ray.rllib.agents import ppo, dqn
from ray import tune
import numpy as np
import torch

import sys
sys.path.append("../..")
from thesis.utils.utils import save_hparams
from thesis.utils.rllib_utils import rllib_ppo_config, setup_ray, save, load
setup_ray(env = "Death")

In [2]:
fleetsize = 6
max_fleetsize = 10
config_args = dict(
    withCollisions = False,
    reward_target = 1, 
    #reward_distance = -0.1,
    reward_block = -1, 
    reward_invalid=-0.1,
    blockTimeout = 120,
    reward_acceptance = 5,
    reward_declined = -0.2,
    dispatchinginterval=240,
    ioquote = 0.8,
    availability = 0.9,
    mttr = 5*60,

)

dir, run = save_hparams(fleetsize = fleetsize, max_fleetsize = max_fleetsize, env_args = config_args)
trainer = ppo.PPOTrainer(rllib_ppo_config(fleetsize, max_fleetsize, config_args, n_stations=5, dispatching=True, with_dispatcher=True, lin_model=True))

2022-10-11 12:21:33,619	WARNING ppo.py:386 -- `train_batch_size` (10000) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=4 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 2500.
2022-10-11 12:21:33,620	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-10-11 12:21:33,621	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_51497_evw37q9d)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2022-10-11 12:21:40,980	WARNING env.py:42 -- Skipping env checking for this experiment


RuntimeError: Found 0 GPUs on your machine (GPU devices found: [])! If your machine
    does not have any GPUs, you should set the config keys `num_gpus` and
    `num_gpus_per_worker` to 0 (they may be set to 1 by default for your
    particular RL algorithm).
To change the config for the `rllib train|rollout` command, use
  `--config={'[key]': '[value]'}` on the command line.
To change the config for `tune.run()` in a script: Modify the python dict
  passed to `tune.run(config=[...])`.
To change the config for an RLlib Trainer instance: Modify the python dict
  passed to the Trainer's constructor, e.g. `PPOTrainer(config=[...])`.


In [ ]:
rllib_ppo_config(fleetsize, max_fleetsize, config_args, n_stations=5, dispatching=True, with_dispatcher=True, lin_model=True)

{'num_workers': 0,
 'num_envs_per_worker': 4,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'complete_episodes',
 'gamma': 0.98,
 'lr': 0.001,
 'train_batch_size': 10000,
 'model': {'_use_default_native_models': False,
  '_disable_preprocessor_api': False,
  '_disable_action_flattening': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,

In [ ]:
#trainer.restore("../../models/Default/6-10-07_10-10_14_51/checkpoint_000300/checkpoint-300")

In [ ]:
#save(trainer, "agv", "../../models/trained_policies/agv_5_9M")

In [ ]:
#load(trainer, "agv", "../../models/trained_policies/test")

In [ ]:
for j in range(500):
    for i in range(20):
        trainer.train()    
    trainer.save(f"{dir}/{run}")

ModelError: ModelError[path=, status={'reason': 'MODEL_ERROR', 'next_event_time': 744.9000000000611, 'episode_count': 130, 'step_count': 833, 'model_time': 744.9000000000611}, error=None, msg='']

: 

In [ ]:
d = trainer.workers.local_worker().get_policy("dispatcher").config

In [ ]:
a=trainer.workers.local_worker().get_policy("agv").config

In [ ]:
# config = rllib_ppo_config(fleetsize, max_fleetsize, config_args, entropy_coeff = 0.1, gamma = 0.8)
# config["kl_coeff"] = 0

In [ ]:
# for j in np.linspace(0.6, 1, 10):
#     config["gamma"] = j
#     dir, run = save_hparams(fleetsize = fleetsize, max_fleetsize = max_fleetsize, env_args = config_args)
#     trainer = ppo.PPOTrainer(config = config)
#     for i in range(10):
#         trainer.train()    
#     trainer.save(f"{dir}/{run}")